In [1]:
import pandas as pd
from openpyxl import load_workbook

# Define file path
file_path = r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Revenue Report Achieve\Revenue Data.xlsx"


# Load Excel file
xls = pd.ExcelFile(file_path)

# Read "Raw Data" and "Category" and 'Sort'sheets
df = pd.read_excel(xls, sheet_name="Revenue", header=0)
df_mapping = pd.read_excel(xls, sheet_name="Mapping", header=0)
#df_sort=pd.read_excel(xls, sheet_name="Sort Table", header=0)
#df['Original States']=df['Region']
df['Etex Customer NO.']=df['Customer']

In [2]:
#----------------Category and Product Group, and MasterData---------------------
df=df.merge(df_mapping, on='P&L Classification', how='left')
#df=df.merge(df_sort, on='P&L Classification', how='left')
df['Category1']=df['Category']

# Read both material and customer master data
material_df = pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\00. Master data\Material Master Data _Etex AU 5500.xlsx", sheet_name=0, header=None)
customer_df = pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\00. Master data\Customer Master Data _Etex.XLSX", sheet_name='Customer Masterdata')

# Set the second row (index 1) as the column headers
material_df.columns = material_df.iloc[1]  # Set second row as header
material_df = material_df.iloc[2:].reset_index(drop=True)  # Drop first two rows

# Pick needed columns and remove duplicates
material_subset = material_df[['Material', 'Item Group','Product Group','Profile Text']] ################################Add
material_subset=material_subset.drop_duplicates(subset=['Material'])
df=df.merge(material_subset,on='Material', how='left')

#Customer MasterData
# Select columns by index
customer_subset = customer_df.iloc[:, [0, 9]]
# Rename columns
customer_subset.columns = ['Customer','Customer Name']
customer_subset=customer_subset.drop_duplicates(subset=['Customer', 'Customer Name'])

#Merge with df
df=df.merge(customer_subset, on='Customer', how='left')

# Drop the extra 'Name Sold_to' column
df = df.drop(columns=['Name Sold_to'])

# in Customer Name, Changes every cell wit Keyword 'BUNNINGS' to 'BUNNINGS GROUP LTD' 
df['Original Customer Name']=df['Customer Name'] ################################Add
df['Customer Name'] = df['Customer Name'].apply(lambda x: 'BUNNINGS GROUP LTD' if 'BUNNINGS' in str(x).upper() else x)

# If 'Customer Name' is in 'BUNNINGS GROUP LTD, CASH SALE, ONE TIME CUSTOMER', set 'Customer' to blank
df.loc[df['Customer Name'] == 'BUNNINGS GROUP LTD', 'Customer'] = ""
df.loc[df['Customer Name'] == 'CASH SALE', 'Customer'] = ""
df.loc[df['Customer Name'] == 'ONE TIME CUSTOMER', 'Customer'] = ""

# Filter only for 'SINIAT' and 'GTEK' brands
df = df[df['Brand'].isin(['SINIAT', 'GTEK'])]


In [3]:
# Parse 'Billing date' as datetime
df['Billing date'] = pd.to_datetime(df['Billing date'], dayfirst=True, errors='coerce')

# Extract Year and Month number as Period
df['Year'] = df['Billing date'].dt.year
df['Period'] = df['Billing date'].dt.month

# Map Month number to Month name
month_map = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
    5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
    9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
df['Month'] = df['Period'].map(month_map)

# Create FiscalPeriod in YYYYMM format
df['FiscalPeriod'] = df['Year'].astype(str) + df['Period'].astype(str).str.zfill(2)

# Add blank columns and YTD
df['Knauf Customer No.'] = ""
df['Knauf Material'] = ""
df['YTD'] = df['Year'].astype(str) + "YTD"



In [4]:
#Confirm Final columns
df1=df[['Category','Bill.doc.','Billing date','Region (Sales Office)','Customer',
       'Customer Name','Plnt','Material','Description','Material Group','Quantity',
       'BUn','Alt Qty 1','Alt UOM 1','Revenue inC Freight','Revenue ex Freight',
       'Freight','P&L Classification','Month','Year','Etex Customer NO.','Region',
       'Category','Knauf Customer No.','Knauf Material','Item Group','YTD',
       'Product Group','Category1','FiscalPeriod']]

df2=df[['Category','Bill.doc.','Billing date','Region (Sales Office)','Customer',
       'Customer Name','Plnt','Material','Description','Material Group','Quantity',
       'BUn','Alt Qty 1','Alt UOM 1','Revenue ex Freight','Revenue inC Freight',
       'Freight','P&L Classification','Month', 'Category','Region','Etex Customer NO.',
       'Original Customer Name','Product Group','Profile Text', 'Brand']]

In [5]:
# Load the workbook
wb = load_workbook(file_path)
ws = wb["Revenue Final"]

# Clear old data but keep headers
ws.delete_rows(2, ws.max_row)  # Deletes from row 2 downwards (keeps headers)

# Write updated DataFrame back to "Raw Data" (including headers)
for c_idx, column in enumerate(df1.columns, start=1):
    ws.cell(row=1, column=c_idx, value=column)  # Write column headers

for r_idx, row in enumerate(df1.itertuples(index=False), start=2):  # Start from row 2 (below headers)
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save the workbook
wb.save(file_path)



In [6]:
# Load the workbook
wb = load_workbook(file_path)
ws = wb["Revenue YTD"]

# Clear old data but keep headers
ws.delete_rows(2, ws.max_row)  # Deletes from row 2 downwards (keeps headers)

# Write updated DataFrame back to "file" (including headers)
for c_idx, column in enumerate(df2.columns, start=1):
    ws.cell(row=1, column=c_idx, value=column)  # Write column headers

for r_idx, row in enumerate(df2.itertuples(index=False), start=2):  # Start from row 2 (below headers)
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save the workbook
wb.save(file_path)
